## Data cleaning

**Table of contents**:
0. Packages and data loading
1. Assigning correct data types
2. Handling missing values
3. Removing outliers
4. Saving clean datasets for Phase 1 and 2


### **0. Packages and data loading**

In [ ]:
# Import packages
import numpy as np
import pandas as pd
import os
from scipy import stats

# Set up the notebook
pd.set_option('display.max_columns', 200)

# Connect to google drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

# Specify a path to the working directory
PROJECT_PATH  = '/content/gdrive/MyDrive/94879_OpAI/final-project'
DATA_PATH     = os.path.join(PROJECT_PATH, 'data')

Mounted at /content/gdrive


In [ ]:
# Load the data and display top-3 rows
df = pd.read_csv(os.path.join(DATA_PATH, 'df.csv'))
df.head(3)

,date,fips,mmwr_week,recip_county,recip_state,completeness_pct,administered_dose1_recip,administered_dose1_pop_pct,administered_dose1_recip_5plus,administered_dose1_recip_5pluspop_pct,administered_dose1_recip_12plus,administered_dose1_recip_12pluspop_pct,administered_dose1_recip_18plus,administered_dose1_recip_18pluspop_pct,administered_dose1_recip_65plus,administered_dose1_recip_65pluspop_pct,series_complete_yes,series_complete_pop_pct,series_complete_5plus,series_complete_5pluspop_pct,series_complete_5to17,series_complete_5to17pop_pct,series_complete_12plus,series_complete_12pluspop_pct,series_complete_18plus,series_complete_18pluspop_pct,series_complete_65plus,series_complete_65pluspop_pct,booster_doses,booster_doses_vax_pct,booster_doses_5plus,booster_doses_5plus_vax_pct,booster_doses_12plus,booster_doses_12plus_vax_pct,booster_doses_18plus,booster_doses_18plus_vax_pct,booster_doses_50plus,booster_doses_50plus_vax_pct,booster_doses_65plus,booster_doses_65plus_vax_pct,second_booster_50plus,second_booster_50plus_vax_pct,second_booster_65plus,second_booster_65plus_vax_pct,svi_ctgy,series_complete_pop_pct_svi,series_complete_5pluspop_pct_svi,series_complete_5to17pop_pct_svi,series_complete_12pluspop_pct_svi,series_complete_18pluspop_pct_svi,series_complete_65pluspop_pct_svi,metro_status,series_complete_pop_pct_ur_equity,series_complete_5pluspop_pct_ur_equity,series_complete_5to17pop_pct_ur_equity,series_complete_12pluspop_pct_ur_equity,series_complete_18pluspop_pct_ur_equity,series_complete_65pluspop_pct_ur_equity,booster_doses_vax_pct_svi,booster_doses_12plusvax_pct_svi,booster_doses_18plusvax_pct_svi,booster_doses_65plusvax_pct_svi,booster_doses_vax_pct_ur_equity,booster_doses_12plusvax_pct_ur_equity,booster_doses_18plusvax_pct_ur_equity,booster_doses_65plusvax_pct_ur_equity,census2019,census2019_5pluspop,census2019_5to17pop,census2019_12pluspop,census2019_18pluspop,census2019_65pluspop,bivalent_booster_5plus,bivalent_booster_5plus_pop_pct,bivalent_booster_12plus,bivalent_booster_12plus_pop_pct,bivalent_booster_18plus,bivalent_booster_18plus_pop_pct,bivalent_booster_65plus,bivalent_booster_65plus_pop_pct,combined_key,population,deaths
0,2020-12-13,1001,51,Autauga County,AL,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B,NaN,NaN,NaN,NaN,NaN,NaN,Metro,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55869.0,NaN,NaN,47574.0,42904.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Autauga, Alabama, US",55869.0,41.0
1,2020-12-14,1001,51,Autauga County,AL,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B,NaN,NaN,NaN,NaN,NaN,NaN,Metro,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55869.0,NaN,NaN,47574.0,42904.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Autauga, Alabama, US",55869.0,41.0
2,2020-12-15,1001,51,Autauga County,AL,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B,NaN,NaN,NaN,NaN,NaN,NaN,Metro,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55869.0,NaN,NaN,47574.0,42904.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Autauga, Alabama, US",55869.0,43.0


In [ ]:
# Check the size of the dataset
df.shape

(1896580, 83)

### **1. Assigning correct data types**

---



In [ ]:
# Check data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1896580 entries, 0 to 1896579
Data columns (total 83 columns):
 #   Column                                   Dtype  
---  ------                                   -----  
 0   date                                     object 
 1   fips                                     int64  
 2   mmwr_week                                int64  
 3   recip_county                             object 
 4   recip_state                              object 
 5   completeness_pct                         float64
 6   administered_dose1_recip                 float64
 7   administered_dose1_pop_pct               float64
 8   administered_dose1_recip_5plus           float64
 9   administered_dose1_recip_5pluspop_pct    float64
 10  administered_dose1_recip_12plus          float64
 11  administered_dose1_recip_12pluspop_pct   float64
 12  administered_dose1_recip_18plus          float64
 13  administered_dose1_recip_18pluspop_pct   float64
 14  administered_dose1

In [ ]:
# Convert 'date' column to a datetime format
df['date'] = pd.to_datetime(df['date'], format='%Y/%m/%d')

# Check the earliest and latest dates
earliest_date = df['date'].min()
latest_date = df['date'].max()

# Print the results
print("Earliest Date:", earliest_date)
print("Latest Date:", latest_date)

Earliest Date: 2020-12-13 00:00:00
Latest Date: 2023-03-08 00:00:00


### **2. Handling missing values**

In [ ]:
# Check the percentage of missing values
percentages = df.isnull().sum() * 100 / df.shape[0]
missing_data = pd.DataFrame({'percent_missing': percentages})
missing_data = missing_data.sort_values(by='percent_missing', ascending=False)
missing_data

,percent_missing
bivalent_booster_12plus,97.128094
bivalent_booster_12plus_pop_pct,97.128094
bivalent_booster_5plus_pop_pct,97.128094
bivalent_booster_5plus,97.128094
bivalent_booster_65plus_pop_pct,97.128094
...,...
fips,0.000000
recip_state,0.000000
recip_county,0.000000
mmwr_week,0.000000


In [ ]:
# Drop columns with more than 50% missing values
threshold = 50
columns_to_drop = missing_data[missing_data['percent_missing'] > threshold].index
df_short = df.drop(columns=columns_to_drop)

# Check the percentage of missing values again
percentages = df_short.isnull().sum() * 100 / df_short.shape[0]
missing_data = pd.DataFrame({'percent_missing': percentages})
missing_data = missing_data.sort_values(by='percent_missing', ascending=False)
missing_data

,percent_missing
series_complete_12pluspop_pct_ur_equity,11.913813
series_complete_12pluspop_pct_svi,11.876114
series_complete_65pluspop_pct_ur_equity,11.136150
series_complete_pop_pct_ur_equity,11.109998
series_complete_18pluspop_pct_ur_equity,11.100033
series_complete_65pluspop_pct_svi,11.066235
series_complete_18pluspop_pct_svi,11.066235
series_complete_pop_pct_svi,11.066235
administered_dose1_recip_12plus,5.903732
administered_dose1_recip_18plus,4.955130


In [ ]:
# For categorical columns, fill missing values with mode of the respective 'fips' group
categorical_columns = ['svi_ctgy', 'metro_status']
for col in categorical_columns:
    df_short[col] = df_short.groupby('fips')[col].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else np.nan))

# For numerical columns, fill missing values with mean of the respective 'fips' group, else fill with the mean of the 'recip_state' group
numerical_columns = df_short.select_dtypes(include=['int64', 'float64']).columns
numerical_columns_with_missing = [col for col in numerical_columns if missing_data.at[col, 'percent_missing'] > 0]
for col in numerical_columns_with_missing:
    df_short[col] = df_short.groupby('fips')[col].transform(lambda x: x.fillna(x.mean()))
    df_short[col] = df_short.groupby('recip_state')[col].transform(lambda x: x.fillna(x.mean()))

# Check the percentage of missing values again
percentages = df_short.isnull().sum() * 100 / df_short.shape[0]
missing_data = pd.DataFrame({'percent_missing': percentages})
missing_data = missing_data.sort_values(by='percent_missing', ascending=False)
missing_data

,percent_missing
series_complete_18pluspop_pct,0.155280
series_complete_yes,0.155280
series_complete_12pluspop_pct_ur_equity,0.155280
series_complete_pop_pct_ur_equity,0.155280
series_complete_65pluspop_pct_svi,0.155280
series_complete_18pluspop_pct_svi,0.155280
series_complete_12pluspop_pct_svi,0.155280
series_complete_pop_pct_svi,0.155280
series_complete_65pluspop_pct,0.155280
series_complete_65plus,0.155280


In [ ]:
# Drop all remaining rows with missing values
df_short = df_short.dropna()

# Check the percentage of missing values again
percentages = df_short.isnull().sum() * 100 / df_short.shape[0]
missing_data = pd.DataFrame({'percent_missing': percentages})
missing_data = missing_data.sort_values(by='percent_missing', ascending=False)
missing_data

,percent_missing
date,0.0
series_complete_pop_pct_ur_equity,0.0
series_complete_65pluspop_pct,0.0
svi_ctgy,0.0
series_complete_pop_pct_svi,0.0
series_complete_12pluspop_pct_svi,0.0
series_complete_18pluspop_pct_svi,0.0
series_complete_65pluspop_pct_svi,0.0
metro_status,0.0
series_complete_12pluspop_pct_ur_equity,0.0


In [ ]:
# Check dataset sizes
print('Original dataset', df.shape)
print('After handling missing values', df_short.shape)

Original dataset (1896580, 83)
After handling missing values (1893046, 38)


### **3. Removing outliers**

In [ ]:
# Calculate Z-scores
z_scores = np.abs(stats.zscore(df_short.loc[:, df_short.columns.difference(['death'])].select_dtypes(include=['float64', 'int64']))) # Check this code
outliers = (z_scores > 3).any(axis=1)

# Separate rows with outliers
outlier_rows = df_short[outliers]

# Calculate the percentage of rows with outliers
outliers_size = outlier_rows.shape[0]/df_short.shape[0]*100
outliers_size

2.0788190038699534

In [ ]:
# Drop the rows with outliers
df_short = df_short[~outliers]

# Check dataset sizes
print('Original dataset', df.shape)
print('After handling missing values and outliers', df_short.shape)

Original dataset (1896580, 83)
After handling missing values and outliers (1853693, 38)


### **4. Saving clean datasets for Phase 1 and 2**

In [ ]:
# Save a full clean dataset
df_short.to_csv(os.path.join(DATA_PATH, 'df_clean.csv'), index=False)

# Filter and save dataset for phase 1 (EDA) - until March 31, 2022
df_phase1_eda = df_short[df_short['date'] <= '2022-03-31']
df_phase1_eda.to_csv(os.path.join(DATA_PATH, 'df_clean_phase1_eda.csv'), index=False)

# Filter and save dataset for phase 1 (Modelling) - until June 30, 2022
df_phase1_modelling = df_short[df_short['date'] <= '2022-06-30']
df_phase1_modelling.to_csv(os.path.join(DATA_PATH, 'df_clean_phase1_modelling.csv'), index=False)

# Filter and save dataset for phase 2 (Modelling) - until September 30, 2022
df_phase2_modelling = df_short[df_short['date'] <= '2022-09-30']
df_phase2_modelling.to_csv(os.path.join(DATA_PATH, 'df_clean_phase2_modelling.csv'), index=False)